In [1]:
import numpy as np 
import pandas as pd
from datetime import datetime

In [2]:
data_1min = pd.read_csv('AAPL_full_1min_adjsplit.csv')
data_1min.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
data_1min['Date'] = pd.to_datetime(data_1min['Date'], format="%Y-%m-%d %H:%M:%S")
data_1hour = pd.read_csv('AAPL_full_1hour_adjsplit.csv')
data_1hour.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
data_1min["Minute Log Return"] = np.log(data_1min['Close']).diff()

In [3]:
#Creates a dataframe with columns "Date" and "Intrahour Volatility"

def calculate_hourly_volatility(dataframe):

    dataframe['Date'] = pd.to_datetime(dataframe['Date'], format="%Y-%m-%d %H:%M:%S")    
    date_log_return_df = dataframe[['Date', 'Minute Log Return']]
    date_log_return_df.set_index('Date', inplace=True)  
    hourly_volatility = date_log_return_df.resample('H').apply(lambda x: (x ** 2).sum(axis=0)).apply(lambda x: x ** 0.5)
    hourly_volatility.reset_index(inplace=True)
    hourly_volatility.columns = ['Date', 'Hourly_Volatility']

    return hourly_volatility


result_df = calculate_hourly_volatility(data_1min)

In [4]:
#Checks that the volatility has been calculated for each data (i.e. the volatility will be greater than zero )

def remove_zero_volatility(dataframe):

   
    filtered_dataframe = dataframe[dataframe['Hourly_Volatility'] != 0]
    return filtered_dataframe

newresults = remove_zero_volatility(result_df)

In [5]:
#Only keeps the rows where the same dates occur in both dataframes

def matching_dataframes_bydate(newresults, data_1hour):

    newresults['Date'] = pd.to_datetime(newresults['Date'])
    data_1hour['Date'] = pd.to_datetime(data_1hour['Date'])
    common_dates = pd.merge(newresults[['Date']], data_1hour[['Date']], on='Date', how='inner') 
    newresults_sync = newresults[newresults['Date'].isin(common_dates['Date'])]
    data_1hour_sync = data_1hour[data_1hour['Date'].isin(common_dates['Date'])]

    return newresults_sync, data_1hour_sync

newresults_sync, data_1hour_sync = matching_dataframes_bydate(newresults, data_1hour)


/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_64652/1521965646.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newresults['Date'] = pd.to_datetime(newresults['Date'])


In [6]:
data_1hour_sync['Date'] = pd.to_datetime(data_1hour_sync['Date'])
newresults_sync['Date'] = pd.to_datetime(newresults_sync['Date'])
data_1hour_sync.set_index('Date', inplace=True)
newresults_sync.set_index('Date', inplace=True)
data_1hour_sync["Hourly Volatility"] = newresults_sync["Hourly_Volatility"]

/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_64652/2840956785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1hour_sync['Date'] = pd.to_datetime(data_1hour_sync['Date'])
/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_64652/2840956785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newresults_sync['Date'] = pd.to_datetime(newresults_sync['Date'])
/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_64652/2840956785.py:5: SettingWithCopyWarning: 
A value is trying

In [7]:
#Developing feature variables

data_1hour_sync['Return'] = np.log(data_1hour_sync['Close']).diff()
data_1hour_sync['Return_Squared'] = np.square(data_1hour_sync['Return'])
data_1hour_sync = data_1hour_sync.dropna()

/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_64652/2430649994.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1hour_sync['Return'] = np.log(data_1hour_sync['Close']).diff()
/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_64652/2430649994.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1hour_sync['Return_Squared'] = np.square(data_1hour_sync['Return'])


In [8]:
pip install pandas_ta

Note: you may need to restart the kernel to use updated packages.


In [9]:
#Developing target variable

data = data_1hour_sync.dropna()
hourly_volatility = data["Hourly Volatility"]
df = pd.DataFrame({'hourly_volatility': hourly_volatility})
df['target'] = np.where(df['hourly_volatility'] > df['hourly_volatility'].shift(1), 1, 0)
df['target'].fillna(0, inplace=True)
data['target'] = df['target']
data_full = data.dropna()
data_full = data_full[["Return_Squared", "Hourly Volatility", "target"]]

In [10]:
data_full

,Return_Squared,Hourly Volatility,target
Date,,,
2005-01-03 10:00:00,6.472128e-05,0.010737,0
2005-01-03 11:00:00,1.807573e-05,0.005680,0
2005-01-03 12:00:00,9.003721e-06,0.004975,0
2005-01-03 13:00:00,6.771714e-05,0.005507,1
2005-01-03 14:00:00,1.887182e-05,0.005832,1
...,...,...,...
2024-01-19 15:00:00,1.334557e-07,0.002568,0
2024-01-19 16:00:00,2.452507e-08,0.001404,0
2024-01-19 17:00:00,5.345758e-07,0.000931,0


In [33]:
data_full.to_csv(r"Intrahour Volatility Dataset.csv")